In [1]:
import pandas as pd

In [2]:
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.0 MB 4.9 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.0 MB 5.4 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 6.2 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.0 MB 6.2 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/12.3 MB 9.1 MB/s eta 0:00:02
   ----------- -----------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = pd.read_csv(r'C:\Users\user\Downloads\classification_logs\training\dataset\synthetic_logs.csv')

In [3]:
df.head()

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [4]:
df

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [6]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [8]:
!pip install sentence-transformers

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.3 MB 3.6 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.3 MB 3.9 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.3 MB 4.1 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.3 MB 4.2 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.3 MB 4.3 MB/s eta 0:00:02
   ------------------------ --------------- 5.8/9.3 MB 4.4 MB/s eta 0:00:01
   ----------------------------- ---------- 6.8/9.3 MB 4.5 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.3 MB 4.5 MB/s eta 0:00:01
   -------------------------------------- - 8.9/9.3 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
    ----------------------------

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings
embeddings = model.encode(df['log_message'], normalize_embeddings=True)

# -------------------------------
# DBSCAN clustering
# -------------------------------
dbscan = DBSCAN(
    eps=0.2,          # distance threshold (tune this)
    min_samples=2,     # minimum points to form a cluster
    metric="cosine"
)

labels = dbscan.fit_predict(embeddings)

# -------------------------------
# Group logs by cluster
# -------------------------------
clusters = {}
for log, label in zip(df['log_message'], labels):
    clusters.setdefault(label, []).append(log)

# -------------------------------
# Print clusters
# -------------------------------
for label, logs in clusters.items():
    if label == -1:
        print("\nNoise / Outliers:")
    else:
        print(f"\nCluster {label}:")
    for log in logs:
        print("  -", log)


c:\Users\user\anaconda3\envs\depthenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



Cluster 0:
  - nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
  - nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
  - nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
  - nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c

In [10]:
df['cluster'] = labels

In [11]:
print(labels)

[0 1 2 ... 0 1 7]


In [8]:
df.head(3)

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2


In [11]:
embeddings[:3]

array([[-0.10293964,  0.03354597, -0.02202606, ...,  0.00457792,
        -0.04259715,  0.00322619],
       [ 0.00804572, -0.03573923,  0.04938739, ...,  0.01538319,
        -0.06230947, -0.02774666],
       [-0.00908224,  0.13003924, -0.05275568, ...,  0.02014104,
        -0.05117098, -0.02930294]], shape=(3, 384), dtype=float32)

In [12]:
df[df.cluster==1]

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,bert,1
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,bert,1
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,bert,1
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,bert,1


In [13]:
cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts>10]

for cluster in large_clusters:
    print(f"Cluster {cluster}:")
    print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print()

Cluster 1017:
Series([], )

Cluster 147:
Series([], )

Cluster 100:
Series([], )

Cluster 86:
Potential vulnerability exploit detected from 1...
Potential vulnerability exploit detected from 1...
Potential vulnerability exploit detected from 1...

Cluster 60:

Cluster 60:

Cluster 58:
UnAuthorized admin access granted to user 6611
User 6734 has gained unauthorized admin access
User 2185 has gained unauthorized admin access
User 1988 has gained unauthorized admin access
UnAuthorized admin access granted to user 8267

Cluster 57:
Vital system component is down: component ID Co...
Vital system component is down: component ID Co...
Vital system component is down: component ID Co...

Cluster 53:
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...
Possible hacking attempt identified from IP 192...

Cluster 52:
Potential security threat: Admin privilege esca...
    Admin privilege escalation alert for user 2893
    Admin privilege escalation a

In [14]:
import re

def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }

    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message,re.IGNORECASE):
            return label

    return None

In [15]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)

In [11]:
df.head()

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [12]:
df

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


In [16]:
df[df.regex_label.notnull()].shape

(500, 7)

In [17]:
df.shape

(2410, 7)

In [18]:
df_non_regex = df[df['regex_label'].isnull()].copy()

In [25]:
df_non_regex['source'].unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [19]:
df_non_regex.shape

(1910, 7)

In [20]:
df_non_legacy = df_non_regex[df_non_regex.source!='LegacyCRM']

In [21]:
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI'], dtype=object)

In [22]:

filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist(), normalize_embeddings=True)

In [32]:
from sklearn.metrics import classification_report

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    filtered_embeddings,
    df_non_legacy['target_label'],
    test_size=0.2,
    random_state=42,
    stratify=df_non_legacy['target_label']   # recommended for classification
)

# Train logistic regression model
model = LogisticRegression(
    random_state=42,
    max_iter=1000
)

model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)


                precision    recall  f1-score   support

Critical Error       1.00      1.00      1.00        32
         Error       1.00      1.00      1.00        36
   HTTP Status       1.00      1.00      1.00       204
Resource Usage       1.00      1.00      1.00        35
Security Alert       1.00      1.00      1.00        74

      accuracy                           1.00       381
     macro avg       1.00      1.00      1.00       381
  weighted avg       1.00      1.00      1.00       381



In [34]:
import joblib

In [36]:
joblib.dump(model, r'C:\Users\user\Downloads\classification_logs\training\model\logistic_regression_model.joblib')

['C:\\Users\\user\\Downloads\\classification_logs\\training\\model\\logistic_regression_model.joblib']